In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from EDMtools import *
from SampTrajsTools import *
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
dim = 2 #dimension
n_anchors = 5 #number of anchors
n_positions = 20 #number of robot sample positions
n_complexity = 4 #model complexity

In [ ]:
np.random.seed(1)
coeffs = 5 * np.random.rand(dim, n_complexity)
anchors = 10 * np.random.rand(dim, n_anchors)


k=np.reshape(range(n_complexity), [n_complexity, 1])
n=np.reshape(range(n_positions), [1, n_positions])
basis = np.cos(np.pi * k * n / n_positions)

trajectory = coeffs @ basis

#continuous trajectory for plotting
n_cont=np.reshape(np.linspace(0, n_positions, 1000), [1, 1000])
basis_cont = np.cos(np.pi * k * n / n_positions)
trajectory_cont = coeffs @ basis_cont

plt.scatter(*anchors, color='blue')
plt.scatter(*trajectory,color='orange')
plt.plot(*trajectory_cont, color='orange')
#plt.axis('off')
#plt.savefig('traj_setup.png')

In [ ]:
#EDM stuff
X = np.hstack([trajectory,anchors])
G = X.T @ X
D = np.outer(np.ones(n_positions + n_anchors),np.diag(G))+np.outer(np.diag(G),np.ones(n_positions+n_anchors))-2*G

In [ ]:
D_tilde = D.copy()
D_tilde[:n_positions,:n_positions] = 0
plt.matshow(D_tilde)
D_topright = D[:n_positions,n_positions:] 

## SDP - based approach
### Noiseless case

In [ ]:
print("Make sure that your cvxpy version is >= 1.0.6!")
import cvxpy
print("Your version:", cvxpy.__version__)

from SampTrajsTools import OPTIONS

# We cane change the global variable OPTIONS here. 

#OPTIONS[cvxpy.SCS]["max_iters"] = 200
# Seems to have no effect: 
#OPTIONS[cvxpy.SCS]["use_indirect"] = False 
# Seems to have no effect either: 
#OPTIONS[cvxpy.SCS]["eps"] = 1e-1
# Seems to have no effect either: 
#OPTIONS[cvxpy.SCS]["scale"] = 1


# Fails completely without this:
OPTIONS[cvxpy.CVXOPT]["kktsolver"] = "robust"

# have no effect:
OPTIONS[cvxpy.CVXOPT]["feastol"] = 1e-1
OPTIONS[cvxpy.CVXOPT]["reltol"] = 1e-1
OPTIONS[cvxpy.CVXOPT]["abstol"] = 1e-1

# leads to faster non-convergence: 
OPTIONS[cvxpy.CVXOPT]["refinement"] = 0


X = semidefRelaxationNoiseless(D_topright, anchors, basis, cvxpy.CVXOPT)
#X = semidefRelaxation(D_topright, anchors, basis)

## MDS - based approach
### Noiseless case

In [ ]:
# find new coefficients
coeffs_est = customMDS(D_topright, basis, anchors)
print(coeffs_est)
print(coeffs)

In [ ]:
trajectory_est = coeffs_est @ basis
trajectory_est_cont = coeffs_est @ basis_cont

plt.scatter(*anchors)
plt.scatter(*trajectory, color='orange', label='true')
plt.plot(*trajectory_cont, color='orange')
plt.scatter(*trajectory_est, color = 'red', label='estimated')
plt.plot(*trajectory_est_cont, color='red')
plt.axis('off')

### Noisy case

In [ ]:
sigma = 4

#state = np.random.RandomState(1)
np.random.seed(1)

D_topright_noisy = D_topright + sigma * np.random.randn(*(D_topright.shape))
coeffs_est_noisy = customMDS(D_topright_noisy, basis, anchors)

coeffs_est_noisy, costs = gradientDescent(
    anchors, basis, coeffs_est_noisy, 
    D_topright_noisy,maxIters=50)
#print(checkStationaryPointSRLS(A,F,C_hat,DTR_tilde))
#plt.plot(costs)

In [ ]:
trajectory_est_noisy = coeffs_est_noisy @ basis
trajectory_est_noisy_cont = coeffs_est_noisy @ basis_cont

plt.scatter(*anchors, color='blue')
plt.plot(*trajectory_cont, color='orange')
plt.scatter(*trajectory_est_noisy, color='red')
plt.plot(*trajectory_est_noisy_cont, color='red')

### Missing measurements

missing measurements between anchors, and between anchors and robot

In [ ]:
np.random.seed(1)

sigma = 2
missing_proportion = 0.9

D_right = D[:,n_positions:]
mask = np.ones(D_right.shape)

mask[:n_positions, :] *= (np.random.rand(n_positions,n_anchors)>missing_proportion)
D_right_missing = mask * D_right

# TODO: why are anchor positions not noisy? 
np.random.seed(1)
D_right_missing[:n_positions,:] += sigma * np.random.randn(n_positions, n_anchors)
D_right_est, errs = alternateGDandKEonDR(D_right_missing, mask, basis, anchors, 
                                    niter=40, DR_true=D_right)
plt.plot(errs)
coeffs_est_missing = customMDS(D_right_est[:n_positions,:], basis, anchors)
#print(C_hat)
#print(C)

In [ ]:
trajectory_est_missing = coeffs_est_missing @ basis
trajectory_est_missing_cont = coeffs_est_missing @ basis_cont

plt.scatter(*anchors, color='blue')
#plt.scatter(*trajectory, color='orange')
plt.plot(*trajectory_cont, color='orange')
plt.scatter(*trajectory_est_missing, color='red')
plt.plot(*trajectory_est_missing_cont, color='red')
plt.axis('off')
#plt.savefig('traj_rec_with_noise.png')

In [ ]:
plt.scatter(*anchors, color='blue')
plt.plot(*trajectory_cont, color='orange')
trajectory_useful = trajectory[:,np.any(mask[:n_positions,:]!=0,axis=1)]
plt.scatter(*trajectory_useful, color='orange')
#plt.axis('off')
#plt.savefig('traj_useful.png')

In [ ]:
plt.figure()
plt.scatter(*anchors)
plt.plot(*trajectory_cont, color='orange')
for i in range(n_positions):
    point = trajectory[:, i]
    if np.sum(mask[i,:])==1:
        plt.scatter(*point,color='orange')
    if np.sum(mask[i,:])==2:
        plt.scatter(*point,color='red')
    if np.sum(mask[i,:])>2:
        plt.scatter(*point,color='green')
plt.scatter(-2.8,6)
plt.scatter(-2.8,5,color='orange')
plt.scatter(-2.8,4,color='red')
plt.scatter(-2.8,3,color='green')
#plt.axis('off')
#plt.legend()
#plt.savefig('traj_useful_colorcoded.png')

In [ ]:
D_right_est, errs = alternateGDandKEonDR(D_right_missing, 
                                    mask, basis, anchors, 
                                    niter=30, DR_true=D_right)
plt.plot(errs)
coeffs_est = customMDS(D_right_est[:n_positions,:], basis, anchors)

In [ ]:
trajectory_est = coeffs_est @ basis
trajectory_est_cont = coeffs_est @ basis_cont

plt.scatter(*anchors)
plt.scatter(*trajectory,color='orange')
plt.plot(*trajectory_cont,color='orange')

plt.scatter(*trajectory_est,color='red')
plt.plot(*trajectory_est_cont,color='red')
#plt.axis('off')
#plt.savefig('traj_rec.png')

In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
tmp = D_right[:n_positions,:]
print(tmp[mask[:n_positions,:]==1])
print(mask[:n_positions,:]*tmp)

tmp = D_right_missing[:n_positions,:]
print(tmp[mask[:n_positions,:]==1])
print(mask[:n_positions,:]*tmp)